In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import h5py
import cv2

In [2]:
def read_many_hdf5(name):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(f"{name}.h5", "r+")

    images = np.array(file["/images"])#.astype("uint8")
    labels = np.array(file["/meta"])#.astype("uint8")
    return images, labels


In [3]:
X,y =  read_many_hdf5('aadhaar_images')

In [4]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [5]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
#model.add(layers.Dense(1024, activation='relu',kernel_initializer='glorot_normal',kernel_regularizer='l2'))
#model.add(layers.Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer='l2'))
#model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer='l2'))
model.add(layers.Dense(1, activation='sigmoid',kernel_initializer='he_normal'))

In [6]:
conv_base.trainable = False

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 21,137,729
Trainable params: 6,423,041
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [9]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[f1])

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
datagen = ImageDataGenerator( rescale=1./255,   
        zoom_range = 0.05,  
        width_shift_range=0.2,  
        height_shift_range=0.2,  
        horizontal_flip=False,  
        vertical_flip=False,
        shear_range=0.05
        
                        )

In [16]:
model.fit(datagen.flow(X,y,shuffle=False),epochs=2)

Epoch 1/2
8/8 [==============================] - 5s 574ms/step - loss: 0.1704 - f1: 0.9669
Epoch 2/2
8/8 [==============================] - 5s 584ms/step - loss: 0.1749 - f1: 0.9470


In [19]:
#model.save('my_model1.h5')



In [17]:
cap = cv2.VideoCapture(0)
count=0
while True:
    ret, frame = cap.read()
    img = frame
    
    frame = cv2.resize(frame,(224,224),interpolation=cv2.INTER_AREA)
    frame = frame /255
    frame = frame.reshape(1,224,224,3)
    output = model.predict_classes(frame)
    if output == [[1]]:
        img = cv2.putText(img,'Found', (30,30) , cv2.FONT_HERSHEY_COMPLEX , 1  , (0,255,0) , 2)
        cv2.imshow('webcam',img)
        print(count,end=' ')
        count+=1
        #continue
    if cv2.waitKey(1) == 13:
        break
    cv2.putText(img,'No Aadhaar Card Found', (30,30) , cv2.FONT_HERSHEY_COMPLEX , 1 , (255,0,0) , 2)
    cv2.imshow('webcam',img)
cap.release()
cv2.destroyAllWindows()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27